In [3]:
import pandas as pd
import warnings
warnings.filterwarnings(action="ignore")
loan=pd.read_csv("loan_data.csv")

In [5]:
print(loan.shape)

(10000, 152)


In [ ]:
# loan_amnt : 대출자의 대출 총액
# loan_status : 대출의 현재 상태
# grade : 대출등급
# int_rate : 대출 이자율
# term : 대출 상품의 기간

In [7]:
# df 변수에 5개 칼럼 담기
df=loan[["loan_amnt", "loan_status", "grade", "int_rate", "term"]]
display(df)

,loan_amnt,loan_status,grade,int_rate,term
0,3600.0,Fully Paid,C,13.99,36 months
1,24700.0,Fully Paid,C,11.99,36 months
2,20000.0,Fully Paid,B,10.78,60 months
3,35000.0,Current,C,14.85,60 months
4,10400.0,Fully Paid,F,22.45,60 months
...,...,...,...,...,...
9995,5000.0,Charged Off,C,13.67,36 months
9996,16000.0,Fully Paid,B,9.76,36 months
9997,24000.0,Current,C,13.18,60 months
9998,10000.0,Fully Paid,B,9.17,36 months


# 대출 기간에 따른 대출 총액
- 대출기간 : term
- 36개월, 60개월

In [19]:
df[["term","loan_amnt"]].groupby("term").sum()

,loan_amnt
term,
36 months,90022925.0
60 months,61021450.0


In [20]:
display(df)

,loan_amnt,loan_status,grade,int_rate,term
0,3600.0,Fully Paid,C,13.99,36 months
1,24700.0,Fully Paid,C,11.99,36 months
2,20000.0,Fully Paid,B,10.78,60 months
3,35000.0,Current,C,14.85,60 months
4,10400.0,Fully Paid,F,22.45,60 months
...,...,...,...,...,...
9995,5000.0,Charged Off,C,13.67,36 months
9996,16000.0,Fully Paid,B,9.76,36 months
9997,24000.0,Current,C,13.18,60 months
9998,10000.0,Fully Paid,B,9.17,36 months


In [29]:
# 대출기간당 총액을 담을 딕셔너리 생성
term_to_loan = {}
# 반복을 돌릴 값 생성
uniq_terms = df['term'].unique()
# for문을 통해 값 생성
for term in uniq_terms:
    term_to_loan_sum = df.loc[df['term']==term , 'loan_amnt'].sum()
    term_to_loan[term] = term_to_loan_sum
# 결과값 리스트에 담아주기
term_to_loan_series = pd.Series(term_to_loan)
# 결과값 DataFrame에 담아주기
result = pd.DataFrame(term_to_loan_series,columns = ['loan_amnt'])
result.index.name = 'term'
result
        

,loan_amnt
term,
36 months,90022925.0
60 months,61021450.0


# 대출 상태가 불량인 사람들의 대출 등급 파악하기
- 대출 상태 : loan_status
- Late(16-30), Late(31-120)


In [30]:
# 대출 상태 컬럼에 있는 데이터 확인
# 불량인 데이터들만 가져오기
# 불량상태 인원들의 등급별 인원수 파악하기
# 오름차순 정렬
display(df)

,loan_amnt,loan_status,grade,int_rate,term
0,3600.0,Fully Paid,C,13.99,36 months
1,24700.0,Fully Paid,C,11.99,36 months
2,20000.0,Fully Paid,B,10.78,60 months
3,35000.0,Current,C,14.85,60 months
4,10400.0,Fully Paid,F,22.45,60 months
...,...,...,...,...,...
9995,5000.0,Charged Off,C,13.67,36 months
9996,16000.0,Fully Paid,B,9.76,36 months
9997,24000.0,Current,C,13.18,60 months
9998,10000.0,Fully Paid,B,9.17,36 months


In [43]:
df_credit_Pung=df[df["loan_status"].str.contains("3")]["grade"]

In [45]:
df_credit_Pung.value_counts().sort_index()

A     3
B    11
C    21
D    10
E    10
F     6
G     1
Name: grade, dtype: int64

In [46]:
# 선생님답
# 대출상태 컬럼에 있는 데이터 확인
loan_status_category = df['loan_status'].unique()
# 불량인 데이터들만 가져오기
bad_status_category = loan_status_category[[4,5]]
# 불량상태 컬럼 추가
df['bad_loan_status'] = df['loan_status'].isin(bad_status_category)
# 불량상태 인원들의 등급별 인원수 파악하기
bad_loan_status_grade = df.loc[df['bad_loan_status'] == True, 'grade'].value_counts()
# 오름차순으로 정렬하기
bad_loan_status_grade.sort_index()

A     3
B    11
C    21
D    10
E    10
F     6
G     1
Name: grade, dtype: int64

# 대출 이자율을 상, 중, 하로 구분하고 구분한 값을 새로운 컬럼을 만들어서 대입하고, 각 요소의 대출액 평균 구하기
- 대출 이자율 : int_rate
- apply cut

In [63]:
df["int_rate"].describe()

count    10000.000000
mean        12.295983
std          4.304762
min          5.320000
25%          9.170000
50%         11.990000
75%         14.850000
max         28.990000
Name: int_rate, dtype: float64

In [48]:
display(df["int_rate"])
display(df)

0       13.99
1       11.99
2       10.78
3       14.85
4       22.45
        ...  
9995    13.67
9996     9.76
9997    13.18
9998     9.17
9999    11.22
Name: int_rate, Length: 10000, dtype: float64

,loan_amnt,loan_status,grade,int_rate,term,bad_loan_status
0,3600.0,Fully Paid,C,13.99,36 months,False
1,24700.0,Fully Paid,C,11.99,36 months,False
2,20000.0,Fully Paid,B,10.78,60 months,False
3,35000.0,Current,C,14.85,60 months,False
4,10400.0,Fully Paid,F,22.45,60 months,False
...,...,...,...,...,...,...
9995,5000.0,Charged Off,C,13.67,36 months,False
9996,16000.0,Fully Paid,B,9.76,36 months,False
9997,24000.0,Current,C,13.18,60 months,False
9998,10000.0,Fully Paid,B,9.17,36 months,False


In [ ]:
def f(df):
    if df["int_rate"]>12:
        return "상"
    elif df["int_rate"]>9:
        return "중"
    else:
        return "하"  
    

In [ ]:
df["이자율 구분"]=df.apply(f, axis=1)

In [ ]:
df[["loan_amnt","이자율 구분"]].groupby("이자율 구분").mean()

In [59]:
# apply함수에 적용할 함수 만들기
def make_int_rate_grade(loan):
    if loan['int_rate'] < 9:
        return 'good'
    elif (loan['int_rate'] >=  9) & (loan['int_rate'] < 12):
        return 'soso'
    else : 
        return 'bad'

# apply함수 적용
df['int_rate_grade1'] = df.apply(make_int_rate_grade, axis = 1)

# 각 요소별 평균 구하기
df[['loan_amnt','int_rate_grade1']].groupby('int_rate_grade1').mean()

,loan_amnt
int_rate_grade1,
bad,16442.674237
good,14340.978399
soso,13767.876429


In [60]:
bins = [5,9,12,29]
label = ['good','soso','bad']
df['int_rate_grade'] = pd.cut(df['int_rate'],bins = bins, labels = label)

grade_term = df['int_rate_grade'].unique()

grade_mean = {}
for grade in grade_term:
    grade_by_mean = df.loc[df['int_rate_grade'] == grade,'loan_amnt'].mean()
    grade_mean[grade] = grade_by_mean

grade_mean

{'bad': 16442.674237363615,
 'soso': 13767.876429479034,
 'good': 14340.978398983481}